In [1]:
from XQuant import BackTestRunner, BackTestOptions, BARRA
from XQuant import Strategy, DataAPI, Config, Tools

In [2]:
DataAPI.get_data('EquDiv')

,secID,ticker,exchangeCD,secShortName,endDate,publishDate,eventProcessCD,perCashDiv,perCashDivAfTax,currencyCD,...,sharesBfDiv,sharesAfDiv,baseShares,perShareTransRatio,bonusShareListDate,frPerCashDiv,frPerCashDivAfTax,frCurrencyCD,bLastTradeDate,perShareDivRatio
0,000559.XSHE,000559,XSHE,万向钱潮,2021-12-31,2022-03-30,6,0.180000,0.180000,CNY,...,3303791344,3303791344,3303791344,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000987.XSHE,000987,XSHE,越秀资本,2021-12-31,2022-04-02,6,0.200000,0.200000,CNY,...,3716394417,5017132462,3716394417,0.35,2022-06-23,NaN,NaN,NaN,NaN,NaN
2,001289.XSHE,001289,XSHE,龙源电力,2021-12-31,2022-03-31,6,0.147000,0.147000,CNY,...,8381963164,8381963164,8381963164,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,002061.XSHE,002061,XSHE,浙江交科,2021-12-31,2022-04-29,6,0.161292,0.161292,CNY,...,1705855834,1705855834,1705855834,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,002243.XSHE,002243,XSHE,力合科创,2021-12-31,2022-03-24,6,0.080000,0.080000,CNY,...,1210604219,1210604219,1210604219,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15207,300286.XSHE,300286,XSHE,安科瑞,2022-12-31,2023-03-18,6,0.200000,0.200000,CNY,...,214716125,214716125,214716125,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15208,301439.XSHE,301439,XSHE,泓淋电力,2022-12-31,2023-04-26,6,0.150000,0.150000,CNY,...,389101809,389101809,389101809,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15209,603688.XSHG,603688,XSHG,石英股份,2022-12-31,2023-03-22,6,0.875000,0.875000,CNY,...,361277126,361277126,361277126,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15210,603703.XSHG,603703,XSHG,盛洋科技,2022-12-31,2023-04-21,6,NaN,NaN,CNY,...,298610000,414982120,290930300,0.40,2023-06-19,NaN,NaN,NaN,NaN,NaN


In [4]:
Config.datatables['EquDiv_info']

{'assets': 'info', 'description': 'DataAPI.EquDivGet'}

In [6]:
Tools.search_keyword('perCashDiv')

2023-08-28 10:51:47.733 | INFO     | XQuant.Utils:search_keyword:608 - 缓存文件被保存至E:\yuankangrui\XQuant\XQuant\Temp\attrs.json


{'perCashDiv': ['EquDiv_info'],
 'frPerCashDiv': ['EquDiv_info'],
 'PE': ['MktEqud', 'MktEqudEval', 'FdmtIndiPSPit'],
 'perCashDivAfTax': ['EquDiv_info'],
 'frPerCashDivAfTax': ['EquDiv_info']}